In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn


seaborn.set_style("darkgrid")
plt.rc("figure", figsize=(16, 6))
plt.rc("savefig", dpi=90)
plt.rc("font", family="sans-serif")
plt.rc("font", size=14)

In [2]:
# Reproducability
import numpy as np
import pandas as pd

gen = np.random.default_rng(23456)
# Common seed used throughout
seed = gen.integers(0, 2**31 - 1)

# The multiple comparison procedures all allow for examining aspects of superior predictive ability. There are three available:

## SPA - The test of Superior Predictive Ability, also known as the Reality Check (and accessible as RealityCheck) or the bootstrap data snooper, examines whether any model in a set of models can outperform a benchmark.

## StepM - The stepwise multiple testing procedure uses sequential testing to determine which models are superior to a benchmark.

## MCS - The model confidence set which computes the set of models which with performance indistinguishable from others in the set.

# All procedures take losses as inputs. That is, smaller values are preferred to larger values. This is common when evaluating forecasting models where the loss function is usually defined as a positive function of the forecast error that is increasing in the absolute error. Leading examples are Mean Square Error (MSE) and Mean Absolute Deviation (MAD).

### The test of Superior Predictive Ability (SPA)

In [7]:
import statsmodels.api as sm
from numpy.random import randn

t = 1000
factors = randn(t, 3)
beta = np.array([1, 0.5, 0.1])
e = randn(t)
y = factors.dot(beta)

In [8]:
# Number of models 修改了噪音的scale，这样就真的会有模型优于benchmark
# 这些模型现在可以看做是不同的模型。
k = 500
model_factors = np.zeros((k, t, 3))
model_losses = np.zeros((500, k))
for i in range(k):
    scale = (2500.0 - i) / 2500.0
    model_factors[i] = factors + scale * randn(1000, 3)
    model_beta = sm.OLS(y[:500], model_factors[i, :500]).fit().params
    # MSE loss
    model_losses[:, i] = (y[500:] - model_factors[i, 500:].dot(model_beta)) ** 2.0

In [9]:
model_losses = pd.DataFrame(model_losses, columns=["model." + str(i) for i in range(k)])
model_losses

,model.0,model.1,model.2,model.3,model.4,model.5,model.6,model.7,model.8,model.9,...,model.490,model.491,model.492,model.493,model.494,model.495,model.496,model.497,model.498,model.499
0,1.000714,0.517129,1.353593,1.117732,0.289636,1.362474,0.231619,0.084221,0.416658,0.007526,...,0.618552,0.571361,0.026694,1.339365,0.120451,0.325398,1.154967,0.259617,0.105253,0.020345
1,0.571927,0.089470,0.001338,0.026757,0.142593,0.043797,1.101484,0.371154,0.096198,0.171014,...,0.145138,0.076619,0.245329,0.000522,1.135519,0.002100,0.036643,0.559780,0.004210,0.004882
2,0.492013,0.584567,2.595898,0.253996,1.357339,0.007615,0.466606,2.167291,2.065312,0.388891,...,2.062427,0.761513,0.701353,2.505941,0.488045,0.006308,0.749271,0.025000,0.709597,0.178667
3,1.307978,0.058019,0.329266,0.283963,0.650220,0.090182,0.000036,0.163929,0.002299,0.690307,...,1.017238,1.401490,0.717295,0.001546,0.098124,0.051504,0.201522,0.152756,0.287464,0.000824
4,0.096110,0.016147,1.902813,3.114043,3.108265,1.406077,0.141238,0.397205,0.778228,0.410645,...,0.164739,0.159720,0.094455,2.298429,0.116643,0.010626,1.338440,0.599864,0.074807,1.004713
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,0.518493,0.123416,0.785964,1.071403,0.280582,0.236427,0.689888,0.118248,1.121349,0.013725,...,0.025673,0.098489,0.076893,0.000097,0.849710,3.980339,0.490648,0.130414,0.000787,0.149982
496,0.461974,0.026853,0.038693,1.222819,0.431046,0.352201,0.387151,0.000047,0.112799,0.087638,...,0.031198,4.294690,0.325498,0.010151,0.100550,0.007473,0.004321,0.042693,0.152952,0.842926
497,0.852324,0.375959,0.091492,0.005675,0.039465,0.166180,0.017791,0.057801,0.101526,0.080867,...,0.776722,0.091306,0.144108,0.006880,0.807186,0.419506,0.288657,0.886053,0.054974,0.373117
498,0.096808,0.053898,0.613433,0.167282,0.020449,1.479776,0.006997,0.055553,0.241790,0.043944,...,0.638699,0.000989,0.293090,0.237976,0.021530,0.027547,0.365743,0.000634,0.422516,0.242936


In [39]:
model_losses.iloc[:,::20].columns

Index(['model.0', 'model.20', 'model.40', 'model.60', 'model.80', 'model.100',
       'model.120', 'model.140', 'model.160', 'model.180', 'model.200',
       'model.220', 'model.240', 'model.260', 'model.280', 'model.300',
       'model.320', 'model.340', 'model.360', 'model.380', 'model.400',
       'model.420', 'model.440', 'model.460', 'model.480'],
      dtype='object')

##  MCS

The Model Confidence Set
The model confidence set takes a set of losses as its input and finds the set which are not statistically different from each other while controlling the familywise error rate. The primary output is a set of p-values, where models with a pvalue above the size are in the MCS. Small p-values indicate that the model is easily rejected from the set that includes the best.

In [28]:
from arch.bootstrap import MCS

losses_mse = pd.read_csv(r'C:\Users\XiaoHuang\AppData\Roaming\MathWorks\MATLAB Add-Ons\Collections\oil_midas\losses_mse.csv',
             header =None,names=['midas_future_rv_mse','midas_future_gepu_mse', 'midas_future_rea_mse','midas_future_double_mse']) #read data 
mcs = MCS(losses_mse, size=0.25, reps=1000, bootstrap='stationary')
mcs.compute()
print("MCS P-values")
print(mcs.pvalues)
print("Included")
included = mcs.included
print(included)
print("Excluded")
excluded = mcs.excluded
print(excluded)

MCS P-values
                         Pvalue
Model name                     
midas_future_gepu_mse     0.226
midas_future_rv_mse       0.228
midas_future_double_mse   0.496
midas_future_rea_mse      1.000
Included
['midas_future_double_mse', 'midas_future_rea_mse']
Excluded
['midas_future_gepu_mse', 'midas_future_rv_mse']


In [13]:
losses_mse

,midas_future_rv_mse,midas_future_gepu_mse,midas_future_rea_mse,midas_future_double_mse
0,-0.011550,-0.011501,-0.011588,-0.011481
1,0.002201,0.002360,0.002168,0.002364
2,-0.000622,-0.000498,-0.000659,-0.000488
3,0.002279,0.002395,0.002242,0.002405
4,0.000726,0.000804,0.000688,0.000819
...,...,...,...,...
1495,0.000255,0.000181,0.000251,0.000174
1496,0.000416,0.000339,0.000411,0.000332
1497,0.000355,0.000275,0.000350,0.000269
1498,0.000203,0.000120,0.000198,0.000114


In [27]:
from arch.bootstrap import MCS

losses_q = pd.read_csv(r'C:\Users\XiaoHuang\AppData\Roaming\MathWorks\MATLAB Add-Ons\Collections\oil_midas\losses_q.csv',
                     header =None,names=['midas_future_rv_q','midas_future_gepu_q', 'midas_future_rea_q','midas_future_double_q']) #read data 
mcs = MCS(losses_q, size=0.25, reps=1000)
mcs.compute()
print("MCS P-values")
print(mcs.pvalues)
print("Included")
included = mcs.included
print(included)
print("Excluded")
excluded = mcs.excluded
print(excluded)

MCS P-values
                       Pvalue
Model name                   
midas_future_rv_q       0.288
midas_future_double_q   0.313
midas_future_gepu_q     0.313
midas_future_rea_q      1.000
Included
['midas_future_double_q', 'midas_future_gepu_q', 'midas_future_rea_q', 'midas_future_rv_q']
Excluded
[]
